In [26]:
class Place:
    def __init__(
            self,
            id,
            place_type: str,
            name,
            address,
            location,
            business_status,
            weekday_descriptions,
            allows_dogs,
            distance=None,
            rating=None,
            google_maps_uri=None,
            photo_resource=None,
            additional_info=None,
    ):
        self.id = id
        self.place_type = place_type
        self.name = name
        self.address = address
        self.location = location
        self.distance = distance
        self.business_status = business_status
        self.weekday_descriptions = weekday_descriptions
        self.allows_dogs = allows_dogs
        self.rating = rating
        self.google_maps_uri = google_maps_uri
        self.photo_resource = photo_resource
        self.additional_info = additional_info


In [31]:
from dotenv import load_dotenv
import os
import requests
import json

load_dotenv()

google_maps_places_url = "https://places.googleapis.com/v1/places"
google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")

X_Goog_FieldMask = (
    "places.id"
    ",places.primaryType"
    ",places.displayName"
    ",places.shortFormattedAddress"
    ",places.location"
    ",places.businessStatus"
    ",places.currentOpeningHours"
    ",places.allowsDogs"
    ",places.rating"
    ",places.googleMapsUri"
    ",places.photos"
)

# X_Goog_FieldMask = (
#     "displayName,"
#     "shortFormattedAddress,"
#     "primaryType,"
#     "location,"
#     "businessStatus,"
#     "currentOpeningHours,"
#     "allowsDogs,"
#     "rating,"
#     "googleMapsUri,"
#     "photos",
# )

def google_map_data_parsing(data):
    display_name = data.get("displayName")
    if display_name is not None:
        display_name = display_name.get("text")

    weekday_descriptions = data.get("currentOpeningHours")
    if weekday_descriptions is not None:
        weekday_descriptions = weekday_descriptions.get("weekdayDescriptions")

    photo_resource = None
    photos = data.get("photos")
    if photos is not None:
        photo_name = photos[0].get("name")
        if photo_name is not None:
            photo_resource = photo_name.split("/")[-1]

    return display_name, weekday_descriptions, photo_resource


def google_maps_places(latitude, longitude, radius):
    url = f"{google_maps_places_url}:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": google_maps_api_key,
        # https://developers.google.com/maps/documentation/places/web-service/nearby-search?_gl=1*xa2znh*_up*MQ..*_ga*MTY3MjAyNjI0MS4xNzIyODU0MDc3*_ga_NRWSTWS78N*MTcyMjg1NDA3Ni4xLjAuMTcyMjg1NDE2MS4wLjAuMA..#fieldmask
        "X-Goog-FieldMask": X_Goog_FieldMask,
    }
    data = {
        # https://developers.google.com/maps/documentation/places/web-service/place-types?_gl=1*sdl46j*_up*MQ..*_ga*MTY3MjAyNjI0MS4xNzIyODU0MDc3*_ga_NRWSTWS78N*MTcyMjg1NDA3Ni4xLjAuMTcyMjg1NDE2MS4wLjAuMA..#table-a
        "includedTypes":  ["pet_store"],  # pet_store
        "maxResultCount": 5,  # max 20
        "languageCode": "ko",
        "rankPreference": "DISTANCE",  # "POPULARITY" or "DISTANCE"
        "locationRestriction": {
            "circle": {
                "center": {"latitude": latitude, "longitude": longitude},
                "radius": radius,
            }
        },
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    print(f"status_code: {response.status_code}")
    response_data = response.json()
    
    place_arr = []
    for place_data in response_data["places"]:
        print(f"place_data: {place_data}")
    
        display_name, weekday_descriptions, photo_resource = google_map_data_parsing(place_data)
        print(f"display_name: {display_name}, weekday_descriptions: {weekday_descriptions}, photo_resource: {photo_resource}")
    
        place_type = "store"
        
        p = Place(
            id=place_data.get("id"),
            place_type=place_type,
            name=display_name,
            address=place_data.get("shortFormattedAddress"),
            location=place_data.get("location"),
            business_status=place_data.get("businessStatus"),
            weekday_descriptions=weekday_descriptions,
            allows_dogs=place_data.get("allowsDogs"),
            rating=place_data.get("rating"),
            google_maps_uri=place_data.get("googleMapsUri"),
            photo_resource=photo_resource,
        )
        
        place_arr.append(p)

    return place_arr

In [34]:
places = google_maps_places(37.26893, 127.0288, 1000)

for place in places:
    print(f"place: {place.__dict__}")

status_code: 200
place_data: {'id': 'ChIJgTPIOFtDezURPZUtNvqsBes', 'location': {'latitude': 37.2666451, 'longitude': 127.0302729}, 'googleMapsUri': 'https://maps.google.com/?cid=16935132164447376701', 'businessStatus': 'OPERATIONAL', 'displayName': {'text': '펫친구', 'languageCode': 'ko'}, 'primaryType': 'pet_store', 'shortFormattedAddress': '수원시 팔달구 인계동 1034-16번지 KR 633호'}
display_name: 펫친구, weekday_descriptions: None, photo_resource: None
place_data: {'id': 'ChIJJziwPURDezUROU9UgsuiuUk', 'location': {'latitude': 37.2686683, 'longitude': 127.0248159}, 'googleMapsUri': 'https://maps.google.com/?cid=5312456230408834873', 'businessStatus': 'OPERATIONAL', 'displayName': {'text': '꼬리치는강아지', 'languageCode': 'ko'}, 'primaryType': 'pet_store', 'shortFormattedAddress': '수원시 팔달구 인계동 번지 1층 975-7 KR'}
display_name: 꼬리치는강아지, weekday_descriptions: None, photo_resource: None
place_data: {'id': 'ChIJvzHAV6pDezURia09Uy8vPKU', 'location': {'latitude': 37.265163, 'longitude': 127.0268385}, 'rating': 5, 'go